In [1]:
import neurokit2 as nk
import mne
import os
import os.path as op
import numpy as np
import pandas as  pd
import matplotlib.pyplot as plt
import re
import numpy
from statsmodels.stats.anova import AnovaRM
from mne.time_frequency import tfr_morlet


**Extracting subjects_ids**

In [2]:
subjects  = [folder for folder in os.listdir('D:\hse\psychodelic_like_experience\subjects') if os.path.isdir(os.path.join('D:\hse\psychodelic_like_experience\subjects', folder))]
print(subjects)

['AS32', 'DE21', 'DF23', 'EE89', 'FK24', 'fx48', 'GF87', 'GG88', 'RD56', 'TR90']


**Extracting answers**

In [ ]:
for subject in subjects:
    data = []

    # Open the file in read mode ('r')
    with open(f'D:/hse/psychodelic_like_experience/subjects/{subject}/{subject}.txt', 'r') as file:
    # Read the file line by line
        for line in file:
            # Use regular expression to extract the condition and number
            match = re.match(r'^([a-zA-Z]+): (\d+)$', line)
            if match:
                # If a match is found, extract filename, condition, and number
                subj = subject  # Replace with the actual filename
                label = match.group(1)
                number = int(match.group(2))

                # Add data to the list
                data.append([subject, label, number])

    # Create a DataFrame from the list of data
    df = pd.DataFrame(data, columns=['Subject', 'Condition', 'Number'])
    df.to_csv(f'D:/hse/psychodelic_like_experience/data_processing/otvety/{subject}.csv', index=False)

    # Now, 'df' contains the desired DataFrame
    print(df)


**Working with events**

In [3]:
#dictionary with codes
all_events_id = {
    'FractalBlue': 101,
    'FractalPurple': 102,
    'FractalGreen': 103,
    'FractalRed': 104,
    'FractalYellow': 105,

    'kaleidoscopeBlue': 201,
    'kaleidoscopePurple': 202,
    'kaleidoscopeGreen': 203,
    'kaleidoscopeRed': 204,
    'kaleidoscopeYellow': 205,

    'CubesControlPinkBlue': 301,
    'CubesControlPurpleGreen': 302,
    'CubesControlBlueRed': 303,
    'CubesControlPinkYellow': 304,
    'CubesControlGreenOrange': 305,

    'HoneyCombBluePink': 401,
    'HoneyCombGreenPurple': 402,
    'HoneyCombRedBlue': 403,
    'HoneyCombOrangePink': 404,
    'HoneyCombGreenOrange': 405
}
event_id = {}

for key, value in all_events_id.items():
    new_key = key.replace("HoneyComb", "HoneyComb/")\
                  .replace("kaleidoscope", "kaleidoscope/")\
                  .replace("Fractal", "Fractal/")\
                  .replace("CubesControl", "CubesControl/")

    event_id[new_key] = value

print(event_id)

{'Fractal/Blue': 101, 'Fractal/Purple': 102, 'Fractal/Green': 103, 'Fractal/Red': 104, 'Fractal/Yellow': 105, 'kaleidoscope/Blue': 201, 'kaleidoscope/Purple': 202, 'kaleidoscope/Green': 203, 'kaleidoscope/Red': 204, 'kaleidoscope/Yellow': 205, 'CubesControl/PinkBlue': 301, 'CubesControl/PurpleGreen': 302, 'CubesControl/BlueRed': 303, 'CubesControl/PinkYellow': 304, 'CubesControl/GreenOrange': 305, 'HoneyComb/BluePink': 401, 'HoneyComb/GreenPurple': 402, 'HoneyComb/RedBlue': 403, 'HoneyComb/OrangePink': 404, 'HoneyComb/GreenOrange': 405}


In [289]:
for subject in subjects:
    conditions = []

    # Open the file in read mode ('r')
    with open(f'D:/hse/psychodelic_like_experience/subjects/{subject}/{subject}.txt', 'r') as file:
        # Read the file line by line
        for line in file:
            # Use regular expression to extract the condition
            match = re.match(r'^([a-zA-Z]+)', line)
            if match:
                # If a match is found, add the label to the list
                conditions.append(match.group(1))

    # Now, 'conditions' contains the extracted conditions
    print(conditions)
    labels = [all_events_id.get(condition, None) for condition in conditions]

    # Print the list of labels
    print(labels)
    raw = mne.io.read_raw_brainvision(f'D:/hse/psychodelic_like_experience/subjects/{subject}/{subject}.vhdr', preload = True)
    mark_array = raw.pick('mark').get_data().flatten()
    events = nk.events_find(event_channel = mark_array, threshold='auto', threshold_keep='below', start_at=0, end_at=None, duration_min=0, duration_max=None, inter_min=0, discard_first=1, discard_last=1, event_labels=labels, event_conditions=conditions)
    events
    events_mne = np.column_stack((events['onset'], np.zeros_like(events['onset']), events['label']))
    events_mne
    mne.write_events(f'D:/hse/psychodelic_like_experience/data_processing/events/{subject}_eve.txt', events_mne, overwrite=True, verbose=None)
    #nk.events_plot(events, mark_array)
    

['FractalBlue', 'CubesControlGreenOrange', 'HoneyCombBluePink', 'kaleidoscopeRed', 'HoneyCombGreenPurple', 'CubesControlPinkBlue', 'kaleidoscopeBlue', 'FractalYellow', 'HoneyCombOrangePink', 'FractalPurple', 'HoneyCombRedBlue', 'FractalGreen', 'kaleidoscopePurple', 'CubesControlPinkYellow', 'HoneyCombGreenOrange', 'kaleidoscopeYellow', 'FractalRed', 'CubesControlPurpleGreen', 'kaleidoscopeGreen', 'CubesControlBlueRed']
[101, 305, 401, 204, 402, 301, 201, 105, 404, 102, 403, 103, 202, 304, 405, 205, 104, 302, 203, 303]
Extracting parameters from D:/hse/psychodelic_like_experience/subjects/AS32/AS32.vhdr...
Setting channel info structure...
Reading 0 ... 797474  =      0.000 ...  1594.948 secs...
Overwriting existing file.
['FractalBlue', 'kaleidoscopeRed', 'FractalRed', 'kaleidoscopeGreen', 'FractalYellow', 'HoneyCombRedBlue', 'CubesControlPurpleGreen', 'kaleidoscopePurple', 'HoneyCombGreenPurple', 'CubesControlGreenOrange', 'kaleidoscopeYellow', 'FractalPurple', 'CubesControlPinkYellow

KeyboardInterrupt: 

**Now we can create epochs...**

In [4]:
epochs_all = []
for subject in subjects:
    
    raw = mne.io.read_raw(f'D:/hse/psychodelic_like_experience/data_processing/eeg_cleaned_ica/{subject}cleaned_raw.fif', preload = True)
    raw.plot(block = True)
    raw = raw.copy().interpolate_bads(reset_bads=False)
    events = mne.read_events(f'D:/hse/psychodelic_like_experience/data_processing/events/{subject}_eve.txt')
    epochs = mne.Epochs(raw, events, event_id, tmin=-5, tmax=20, baseline=None, preload = True)
    epochs.plot( n_epochs=1, n_channels=20, title=None, show=True, decim='auto', block = True)
    epochs_all.append(epochs)

Opening raw data file D:/hse/psychodelic_like_experience/data_processing/eeg_cleaned_ica/AS32cleaned_raw.fif...
    Read a total of 2 projection items:
        EOG-eeg--0.200-0.200-PCA-01 (1 x 28)  idle
        EOG-eeg--0.200-0.200-PCA-02 (1 x 28)  idle
    Range : 0 ... 797474 =      0.000 ...  1594.948 secs
Ready.
Reading 0 ... 797474  =      0.000 ...  1594.948 secs...
Using qt as 2D backend.
Channels marked as bad:
['FC2']
Interpolating bad channels
    Automatic origin fit: head of radius 94.6 mm
Computing interpolation matrix from 27 sensor positions
Interpolating 1 sensors
Not setting metadata
20 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 2)
2 projection items activated
Using data from preloaded Raw for 20 events and 12501 original time points ...
0 bad epochs dropped


C:\Users\User\AppData\Local\Temp\ipykernel_9716\386509325.py:9: FutureWarning: The current default events=None is deprecated and will change to events=True in MNE 1.6. Set events=False to suppress this warning.
  epochs.plot( n_epochs=1, n_channels=20, title=None, show=True, decim='auto', block = True)


Dropped 12 epochs: 1, 2, 7, 10, 11, 12, 13, 14, 15, 16, 17, 19
The following epochs were marked as bad and are dropped:
[1, 2, 7, 10, 11, 12, 13, 14, 15, 16, 17, 19]
Channels marked as bad:
['FC2']
Opening raw data file D:/hse/psychodelic_like_experience/data_processing/eeg_cleaned_ica/DE21cleaned_raw.fif...
    Read a total of 2 projection items:
        EOG-eeg--0.200-0.200-PCA-01 (1 x 28)  idle
        EOG-eeg--0.200-0.200-PCA-02 (1 x 28)  idle
    Range : 0 ... 802799 =      0.000 ...  1605.598 secs
Ready.
Reading 0 ... 802799  =      0.000 ...  1605.598 secs...


C:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyqtgraph\GraphicsScene\GraphicsScene.py:378: RuntimeWarning: Projection vector 'EOG-eeg--0.200-0.200-PCA-02' has been reduced to 88.89% of its original magnitude by subselecting 27/28 of the original channels. If the ignored channels were bad during SSP computation, we recommend recomputing proj (via compute_proj_raw or related functions) with the bad channels properly marked, because computing SSP with bad channels present in the data but unmarked is dangerous (it can bias the PCA used by SSP). On the other hand, if you know that all channels were good during SSP computation, you can safely use info.normalize_proj() to suppress this warning during projection.
  item.mouseClickEvent(ev)
C:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyqtgraph\GraphicsScene\GraphicsScene.py:378: RuntimeWarning: Projection vector 'EOG-eeg--0.200-0.200-PCA-02' has been reduced to 88.89% of its original magnit

Channels marked as bad:
['Fp1', 'Fp2']
Interpolating bad channels
    Automatic origin fit: head of radius 94.6 mm
Computing interpolation matrix from 26 sensor positions
Interpolating 2 sensors
Not setting metadata
20 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 2)
2 projection items activated
Using data from preloaded Raw for 20 events and 12501 original time points ...
0 bad epochs dropped


...

Dropped 6 epochs: 5, 6, 12, 15, 16, 18
The following epochs were marked as bad and are dropped:
[5, 6, 12, 15, 16, 18]
Channels marked as bad:
['Fp1', 'Fp2', 'F8']
Opening raw data file D:/hse/psychodelic_like_experience/data_processing/eeg_cleaned_ica/TR90cleaned_raw.fif...
    Read a total of 2 projection items:
        EOG-eeg--0.200-0.200-PCA-01 (1 x 28)  idle
        EOG-eeg--0.200-0.200-PCA-02 (1 x 28)  idle
    Range : 0 ... 799324 =      0.000 ...  1598.648 secs
Ready.
Reading 0 ... 799324  =      0.000 ...  1598.648 secs...
Channels marked as bad:
none
Not setting metadata
20 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 2)
2 projection items activated
Using data from preloaded Raw for 20 events and 12501 original time points ...


C:\Users\User\AppData\Local\Temp\ipykernel_9716\386509325.py:6: RuntimeWarning: No bad channels to interpolate. Doing nothing...
  raw = raw.copy().interpolate_bads(reset_bads=False)


0 bad epochs dropped


C:\Users\User\AppData\Local\Temp\ipykernel_9716\386509325.py:9: FutureWarning: The current default events=None is deprecated and will change to events=True in MNE 1.6. Set events=False to suppress this warning.
  epochs.plot( n_epochs=1, n_channels=20, title=None, show=True, decim='auto', block = True)


Dropped 1 epoch: 19
The following epochs were marked as bad and are dropped:
[19]
Channels marked as bad:
none


**...and investigate their time-frequency properties**

In [252]:
scenes = ['kaleidoscope', 'CubesControl', 'HoneyComb', 'Fractal']
decim = 5
freqs = np.arange(4, 30, 1)  # define frequencies of interest
n_cycles = freqs / freqs[0]
for scene in scenes:
    list_epochs = []
    for i in range(len(epochs_all)):
        list_epochs.append(epochs_all[i][scene])
    tfr = []
    for k in list_epochs:
            k.interpolate_bads(reset_bads=True, mode='accurate', origin='auto', method=dict(eeg="spline"), exclude=(), verbose=None)
            tfr_epochs = tfr_morlet(
            k,
            freqs,
            n_cycles=n_cycles,
            decim=decim,
            average=False,
            return_itc=False,
            n_jobs=None,
            )
            #.average(method='mean', dim='epochs').to_data_frame(long_format = True)
            tfr.append(tfr_epochs)
    df_tfr = []
    for i in range(0, len(tfr)):
       df = tfr[i].average(method='mean', dim='epochs').to_data_frame(long_format = True)
       df_tfr.append(df)
    df_tfr = pd.concat(df_tfr)
    df_tfr.groupby(['freq', 'channel'])['value'].mean()
    #df_kaleidoscop_tfr = pd.concat(df_kaleidoscop_tfr)
    theta = df_tfr.query('freq > 3 and freq < 9').groupby(['freq', 'channel'])['value'].mean()
    #theta.to_csv(f'D:/hse/psychodelic_like_experience/data_processing/tfr_epochs/{scene}/theta.csv')
    alpha = df_tfr.query('freq > 7 and freq < 13').groupby(['freq', 'channel'])['value'].mean()
    #alpha.to_csv(f'D:/hse/psychodelic_like_experience/data_processing/tfr_epochs/{scene}/alpha.csv')
    beta = df_tfr.query('freq > 12 and freq < 31').groupby(['freq', 'channel'])['value'].mean()
    #beta.to_csv(f'D:/hse/psychodelic_like_experience/data_processing/tfr_epochs/{scene}/beta.csv')
    t = theta.groupby(['channel']).mean()
    a = alpha.groupby(['channel']).mean()
    b = beta.groupby(['channel']).mean()
    temp.data = np.array([t,a,b]).T* (10**10)
    numpy.savetxt(f'D:/hse/psychodelic_like_experience/data_processing/tfr_epochs/{scene}.csv', temp.data, delimiter=",")  
    fig = temp.plot_topomap(times=temp.times,  ch_type='eeg', scalings = 1, units = '', show = False,  time_unit='s',  colorbar = False, extrapolate = "head")
    fig.suptitle(f'{scene}',  fontsize=20, fontweight='bold')
    titles = ['theta', 'alpha', 'beta']
    # Set titles for subplots
    for ax, title in zip(fig.axes, titles):
        ax.set_title(title)
    fig
    fig.savefig(f'D:/hse/psychodelic_like_experience/data_processing/tfr_epochs/{scene}/pic.png')


Interpolating bad channels
    Automatic origin fit: head of radius 94.6 mm
Computing interpolation matrix from 27 sensor positions
Interpolating 1 sensors


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    2.3s


Not setting metadata
Interpolating bad channels
    Automatic origin fit: head of radius 94.6 mm
Computing interpolation matrix from 26 sensor positions
Interpolating 2 sensors


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    3.7s


Not setting metadata


...

Not setting metadata
Interpolating bad channels
    Automatic origin fit: head of radius 94.6 mm
Computing interpolation matrix from 25 sensor positions
Interpolating 3 sensors


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    3.1s


Not setting metadata


C:\Users\User\AppData\Local\Temp\ipykernel_9716\3786098942.py:8: RuntimeWarning: No bad channels to interpolate. Doing nothing...
  k.interpolate_bads(reset_bads=True, mode='accurate', origin='auto', method=dict(eeg="spline"), exclude=(), verbose=None)
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    4.0s


Not setting metadata
Converting "channel" to "category"...
Converting "ch_type" to "category"...
Converting "channel" to "category"...
Converting "ch_type" to "category"...
Converting "channel" to "category"...
Converting "ch_type" to "category"...
Converting "channel" to "category"...
Converting "ch_type" to "category"...
Converting "channel" to "category"...
Converting "ch_type" to "category"...
Converting "channel" to "category"...
Converting "ch_type" to "category"...
Converting "channel" to "category"...
Converting "ch_type" to "category"...
Converting "channel" to "category"...
Converting "ch_type" to "category"...
Converting "channel" to "category"...
Converting "ch_type" to "category"...
Converting "channel" to "category"...
Converting "ch_type" to "category"...


**Run the simpliest ANOVA just for preliminary analysis**

In [253]:
kdata = pd.read_csv(f'D:/hse/psychodelic_like_experience/data_processing/tfr_epochs/kaleidoscope.csv', header = None)
cdata =  pd.read_csv(f'D:/hse/psychodelic_like_experience/data_processing/tfr_epochs/CubesControl.csv', header = None)
hdata = pd.read_csv(f'D:/hse/psychodelic_like_experience/data_processing/tfr_epochs/HoneyComb.csv', header = None)
fdata = pd.read_csv(f'D:/hse/psychodelic_like_experience/data_processing/tfr_epochs/Fractal.csv', header = None)

In [287]:
data = {'sensor': np.arange(0, 28),
        'theta': kdata[1].values,
        'condition': 'kaleidoscope'}
k_df = pd.DataFrame(data)
data = {'sensor': np.arange(0, 28),
        'theta': hdata[1].values,
        'condition': 'honeycomb'}
h_df = pd.DataFrame(data)
data = {'sensor': np.arange(0, 28),
        'theta': hdata[1].values,
        'condition': 'fractal'}
f_df = pd.DataFrame(data)
data = {'sensor': np.arange(0, 28),
        'theta': hdata[1].values,
        'condition': 'cubescontrol'}
#c_df = pd.DataFrame(data)
#df_for_stats = pd.DataFrame()
#df_for_stats = df_for_stats.concat(k_df)#.concat(h_df).concat(f_df).concat(c_df)
#df_for_stats = pd.concat(k_df, h_df, f_df, c_df )

In [284]:
df = pd.concat(pd.DataFrame(i) for i in (k_df, h_df, f_df, c_df))

In [285]:
df

,sensor,theta,condition
0,0,5.686399,kaleidoscope
1,1,6.787471,kaleidoscope
2,2,8.717342,kaleidoscope
3,3,8.698775,kaleidoscope
4,4,5.063024,kaleidoscope
...,...,...,...
23,23,6.288982,cubescontrol
24,24,7.650463,cubescontrol
25,25,10.637399,cubescontrol
26,26,2.764090,cubescontrol


In [288]:
aovrm = AnovaRM(df, 'theta', 'sensor', within=['condition'])
res = aovrm.fit()

print(res)


                 Anova
          F Value Num DF  Den DF Pr > F
---------------------------------------
condition  0.5250 3.0000 81.0000 0.6663



In [244]:
h_c = hdata.subtract(cdata) #to look more precisely at difference between euclidian and non-euclidian scenes

In [228]:
temp.data = h_c.values

In [231]:
    fig = temp.plot_topomap(times=temp.times,  ch_type='eeg', scalings = 1, units = 'e-10', show = False,  time_unit='s',  colorbar = True, extrapolate = "head")
    fig.suptitle('HoneyComb - CubesControl',  fontsize=16, fontweight='bold')
    titles = ['theta', 'alpha', 'beta']
    # Set titles for subplots
    for ax, title in zip(fig.axes, titles):
        ax.set_title(title)
    fig
    fig.savefig(f'D:/hse/psychodelic_like_experience/data_processing/pics/honey_cubes.png')

**SPECTRAL CONNECTIVITY**

In [ ]:
from mne_connectivity import spectral_connectivity_epochs
from mne_connectivity.viz import plot_sensors_connectivity

In [ ]:
raw = mne.io.read_raw_brainvision('C:/Users/User/Downloads/FX48.vhdr', preload = True)
info = mne.create_info(ch_names = raw.pick('eeg').ch_names, sfreq = 500, ch_types='eeg', verbose=None)

In [464]:
scenes = ['kaleidoscope', 'CubesControl', 'HoneyComb', 'Fractal']
#scenes = ['Fractal']
for scene in scenes:
    list_epochs = []
    for i in range(len(epochs_all)):
        list_epochs.append(epochs_all[i][scene])
    Fractal = []
    for i in range(1, len(list_epochs)):
        epochs = list_epochs[i].pick('eeg').get_data()
        for j in range(0, epochs.shape[0]):
            epoch = epochs[j]
            #print(epoch.shape)
            Fractal.append(epoch)
    data = np.array(Fractal)       
    data = mne.EpochsArray(data, info, events=None, tmin=0, proj=True)
    data.set_montage(montage='standard_1020')
    fmin, fmax = 4., 9.
    sfreq = 500 
    tmin = 0.0 
    data.load_data().pick('eeg') 
    con = spectral_connectivity_epochs(
        data, method='wpli', mode='multitaper', sfreq=sfreq, fmin=fmin, fmax=fmax,
        faverage=True, tmin=tmin, mt_adaptive=False, n_jobs=1)
    connectivity_fig = plot_sensors_connectivity(data.info, con.get_data(output="dense")[:, :, 0], picks='eeg', cbar_label=f'Connectivity_{scene}')
    screenshot = connectivity_fig.plotter.screenshot()

    # The screenshot is just a NumPy array, so we can display it via imshow()
    # and then save it to a file.
    fig, ax = plt.subplots(figsize=(10, 10))
    ax.imshow(screenshot, origin='upper')
    ax.set_axis_off()  # Disable axis labels and ticks
    fig.tight_layout()
    fig.savefig(f'D:/hse/psychodelic_like_experience/data_processing/connectivity/connectivity_{scene}.png', dpi=600)

Not setting metadata
36 matching events found
No baseline correction applied
0 projection items activated
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 378 connections
    using t=0.000s..25.000s for estimation (12501 points)
    frequencies: 4.0Hz..9.0Hz (125 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: WPLI
    computing connectivity for epoch 1


C:\Users\User\AppData\Local\Temp\ipykernel_9716\1958929368.py:21: RuntimeWarning: There were no Annotations stored in <EpochsArray |  36 events (all good), 0 – 25 s, baseline off, ~96.2 MB, data loaded,
 '1': 36>, so metadata was not modified.
  con = spectral_connectivity_epochs(


    computing connectivity for epoch 2
    computing connectivity for epoch 3
    computing connectivity for epoch 4
    computing connectivity for epoch 5
    computing connectivity for epoch 6
    computing connectivity for epoch 7
    computing connectivity for epoch 8
    computing connectivity for epoch 9
    computing connectivity for epoch 10
    computing connectivity for epoch 11
    computing connectivity for epoch 12
    computing connectivity for epoch 13
    computing connectivity for epoch 14
    computing connectivity for epoch 15
    computing connectivity for epoch 16
    computing connectivity for epoch 17
    computing connectivity for epoch 18
    computing connectivity for epoch 19
    computing connectivity for epoch 20
    computing connectivity for epoch 21
    computing connectivity for epoch 22
    computing connectivity for epoch 23
    computing connectivity for epoch 24
    computing connectivity for epoch 25
    computing connectivity for epoch 26
    comp

C:\Users\User\AppData\Local\Temp\ipykernel_9716\1958929368.py:21: RuntimeWarning: There were no Annotations stored in <EpochsArray |  35 events (all good), 0 – 25 s, baseline off, ~93.5 MB, data loaded,
 '1': 35>, so metadata was not modified.
  con = spectral_connectivity_epochs(


    computing connectivity for epoch 2
    computing connectivity for epoch 3
    computing connectivity for epoch 4
    computing connectivity for epoch 5
    computing connectivity for epoch 6
    computing connectivity for epoch 7
    computing connectivity for epoch 8
    computing connectivity for epoch 9
    computing connectivity for epoch 10
    computing connectivity for epoch 11
    computing connectivity for epoch 12
    computing connectivity for epoch 13
    computing connectivity for epoch 14
    computing connectivity for epoch 15
    computing connectivity for epoch 16
    computing connectivity for epoch 17
    computing connectivity for epoch 18
    computing connectivity for epoch 19
    computing connectivity for epoch 20
    computing connectivity for epoch 21
    computing connectivity for epoch 22
    computing connectivity for epoch 23
    computing connectivity for epoch 24
    computing connectivity for epoch 25
    computing connectivity for epoch 26
    comp

C:\Users\User\AppData\Local\Temp\ipykernel_9716\1958929368.py:21: RuntimeWarning: There were no Annotations stored in <EpochsArray |  34 events (all good), 0 – 25 s, baseline off, ~90.8 MB, data loaded,
 '1': 34>, so metadata was not modified.
  con = spectral_connectivity_epochs(


    computing connectivity for epoch 2
    computing connectivity for epoch 3
    computing connectivity for epoch 4
    computing connectivity for epoch 5
    computing connectivity for epoch 6
    computing connectivity for epoch 7
    computing connectivity for epoch 8
    computing connectivity for epoch 9
    computing connectivity for epoch 10
    computing connectivity for epoch 11
    computing connectivity for epoch 12
    computing connectivity for epoch 13
    computing connectivity for epoch 14
    computing connectivity for epoch 15
    computing connectivity for epoch 16
    computing connectivity for epoch 17
    computing connectivity for epoch 18
    computing connectivity for epoch 19
    computing connectivity for epoch 20
    computing connectivity for epoch 21
    computing connectivity for epoch 22
    computing connectivity for epoch 23
    computing connectivity for epoch 24
    computing connectivity for epoch 25
    computing connectivity for epoch 26
    comp

C:\Users\User\AppData\Local\Temp\ipykernel_9716\1958929368.py:21: RuntimeWarning: There were no Annotations stored in <EpochsArray |  34 events (all good), 0 – 25 s, baseline off, ~90.8 MB, data loaded,
 '1': 34>, so metadata was not modified.
  con = spectral_connectivity_epochs(


    computing connectivity for epoch 2
    computing connectivity for epoch 3
    computing connectivity for epoch 4
    computing connectivity for epoch 5
    computing connectivity for epoch 6
    computing connectivity for epoch 7
    computing connectivity for epoch 8
    computing connectivity for epoch 9
    computing connectivity for epoch 10
    computing connectivity for epoch 11
    computing connectivity for epoch 12
    computing connectivity for epoch 13
    computing connectivity for epoch 14
    computing connectivity for epoch 15
    computing connectivity for epoch 16
    computing connectivity for epoch 17
    computing connectivity for epoch 18
    computing connectivity for epoch 19
    computing connectivity for epoch 20
    computing connectivity for epoch 21
    computing connectivity for epoch 22
    computing connectivity for epoch 23
    computing connectivity for epoch 24
    computing connectivity for epoch 25
    computing connectivity for epoch 26
    comp

**...FIN?**
no, fur